<font size="6">**7 steps to fine-tune a LLM**</font>

In [ ]:
!pip install -q openai langchain pyvis gradio newspaper3k chromadb==0.4.3 tiktoken datasets huggingface huggingface_hub evaluate accelerate transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing 

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
import pandas as pd
import numpy as np

**Requirements**

For this tutorial, the following libraries are needed:
- Throughout the whole tutorial, we will be using the `transformers` library.
- For the fine-tuning either `pytorch` or `tensorflow` are required. (This Notebook will be implemented with `pytorch`)
- To push the fine-tuned model to HuggingFace, the `HuggingFace_hub`library is required.

## STEP 1 - Having our concrete objective clear

In [ ]:
from datasets import load_dataset

dataset = load_dataset("FinGPT/fingpt-sentiment-train")
df = pd.DataFrame(dataset['train'])


## STEP 4 - Tokenizer



In [ ]:
# from transformers import GPT2Tokenizer

# # Loading the dataset to train our model
# dataset = load_dataset("mteb/tweet_sentiment_extraction")

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

# def tokenize_function(examples):
#    return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})

In [ ]:
from transformers import GPT2Tokenizer

# Loading the dataset to train our model
dataset = load_dataset("FinGPT/fingpt-sentiment-train")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
   return tokenizer(examples["input"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenizer.save_pretrained('fine_tuned_finsetiment_tokenizer/')

('fine_tuned_finsetiment_tokenizer/tokenizer_config.json',
 'fine_tuned_finsetiment_tokenizer/special_tokens_map.json',
 'fine_tuned_finsetiment_tokenizer/vocab.json',
 'fine_tuned_finsetiment_tokenizer/merges.txt',
 'fine_tuned_finsetiment_tokenizer/added_tokens.json')

In [ ]:
from datasets import DatasetDict
import random
import string

# Function to generate a random ID
def generate_random_id():
  return ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))

# Get all unique values from the 'output' column
unique_outputs = set(dataset['train']['output'])

# Define a mapping from unique output values to numeric labels
#label_mapping = {'mildly positive': 0, 'positive': 1, 'strong positive':2, 'moderately positive': 3, 'negative': 4, 'neutral': 5}  # Add more mappings as needed
label_mapping = {label: index for index, label in enumerate(unique_outputs)}

# Transform the dataset
def transform_dataset(dataset):
  # Rename columns
  dataset = dataset.rename_column('input', 'text')
  dataset = dataset.rename_column('output', 'label_text')

  # Remove 'instruction' column
  dataset = dataset.remove_columns(['instruction'])

  # Add 'id' and 'label' columns
  dataset = dataset.add_column('id', [generate_random_id() for _ in range(dataset.num_rows)])
  dataset = dataset.add_column('label', [label_mapping[label_text] for label_text in dataset['label_text']])

  return dataset

# Apply the transformation
transformed_dataset = DatasetDict({'train': transform_dataset(tokenized_datasets['train'])})

ModuleNotFoundError: No module named 'datasets'

In [ ]:
label_mapping

{'mildly positive': 0,
 'mildly negative': 1,
 'moderately negative': 2,
 'moderately positive': 3,
 'positive': 4,
 'negative': 5,
 'neutral': 6,
 'strong negative': 7,
 'strong positive': 8}

In [ ]:
# Set the format of the dataset to reorder columns
transformed_dataset['train'].set_format(type=None, columns=['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'])

# Split the 'train' dataset into new 'train' and 'test' datasets
train_test_split = transformed_dataset['train'].train_test_split(test_size=0.3, seed=42)

# Update 'tokenized_datasets' to include the new 'test' dataset
tokenized_datasets['test'] = train_test_split['test']
tokenized_datasets['train'] = train_test_split['train']

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

NameError: name 'transformed_dataset' is not defined

In [ ]:
small_train_dataset

Dataset({
    features: ['text', 'label_text', 'input_ids', 'attention_mask', 'id', 'label'],
    num_rows: 100
})

## STEP 5 - Initialize our base model

In [1]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=9)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## STEP 6 - Evaluate method


In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)

   return metric.compute(predictions=predictions, references=labels)

ModuleNotFoundError: No module named 'evaluate'

## STEP 7 - Fine-tune using the Trainer Method



In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=75, training_loss=3.5832206217447915, metrics={'train_runtime': 77.7581, 'train_samples_per_second': 3.858, 'train_steps_per_second': 0.965, 'total_flos': 156787959398400.0, 'train_loss': 3.5832206217447915, 'epoch': 3.0})

In [ ]:
trainer.evaluate()
trainer.save_model("fine_tuned_finsetiment_model")


## Sharing the model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Replace with the actual path to your fine-tuned model
model_path = "fine_tuned_finsetiment_model/"

# We get our model
finetuned_model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
finetuned_model.push_to_hub("fine_tuned_financial_setiment_analysis_gpt2_model")

CommitInfo(commit_url='https://huggingface.co/rezahf2024/fine_tuned_financial_setiment_analysis_gpt2_model/commit/20a4283043535f0bb3ed7ace94d772a1a049dfa2', commit_message='Upload GPT2ForSequenceClassification', commit_description='', oid='20a4283043535f0bb3ed7ace94d772a1a049dfa2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("fine_tuned_financial_setiment_analysis_gpt2_model")

CommitInfo(commit_url='https://huggingface.co/rezahf2024/fine_tuned_financial_setiment_analysis_gpt2_model/commit/1b7c35eaf957b498d3b53f6b31fd28ddaadef879', commit_message='Upload tokenizer', commit_description='', oid='1b7c35eaf957b498d3b53f6b31fd28ddaadef879', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import torch

# Load your fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("fine_tuned_finsetiment_model")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_finsetiment_model")

# Define the label mapping as provided
label_mapping_reverse = {
'LABEL_0': 'mildly positive',
'LABEL_1': 'mildly negative',
'LABEL_2': 'moderately negative',
'LABEL_3': 'moderately positive',
'LABEL_4': 'positive',
'LABEL_5': 'negative',
'LABEL_6': 'neutral',
'LABEL_7': 'strong negative',
'LABEL_8': 'strong positive'
}

def model_predict(text):
  # Tokenize the input text
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

  # Get predictions from the model
  with torch.no_grad():
    logits = model(**inputs).logits

    print(logits)

    # Convert to probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Create a list of tuples with label and probability
    label_prob_pairs = [(label_mapping_reverse[label_idx], prob.item())
    for label_idx, prob in enumerate(probabilities.squeeze())]

    # Sort the list by probability in descending order
    sorted_label_prob_pairs = sorted(label_prob_pairs, key=lambda pair: pair[1], reverse=True)

  # Return the sorted list of label-probability pairs
  return sorted_label_prob_pairs

# Example usage
text = "Intel Corporation (NASDAQ: INTC) has unveiled a remote verification platform called Project Amber"
predictions = model_predict(text)
for label, prob in predictions:
  print(f"{label}: {prob:.3f}")

NameError: name 'AutoModelForSequenceClassification' is not defined